# Virus Tracking

The CDC tracks virus cases by state, where each state reports their current count to the CDC.
I want to see the number of cases in Texas as they grow over time, broken down by city if possible.

I have only found 1 website that offers anything like this kind of data, and it only shows the "current" cases:
https://dshs.texas.gov/news/updates.shtm

DSHS is updating their data source every day at noon, so if I add to *my* data source every day at 1pm (rather than replacing the current), I should be able to see change over time.

In [3]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://dshs.texas.gov/news/updates.shtm", verify=False)
source = BeautifulSoup(response.text, "html.parser")

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:849: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [15]:
import pandas as pd
import datetime as dt

tables = source.find_all("table")
county_table = [t for t in tables if t.has_attr("summary") and t.attrs["summary"] == "COVID-19 Cases in Texas Counties"][0]

row_groups = [tr.find_all("td") for tr in county_table.find_all("tr")][1:]

today = dt.datetime.today()

num_cases = [{
    "county": td[0].text,
    "date": f"{today.month}/{today.day}/{today.year}",
    "num_cases": td[1].text
} for td in row_groups]

df_num_cases = pd.DataFrame(num_cases)

df_num_cases.set_index(["county", "date"], inplace=True)
df_num_cases.to_csv("data.csv")

display(df_num_cases)

,,num_cases
county,date,
Bell,3/20/2020,2
Bexar,3/20/2020,12
Bowie,3/20/2020,1
Brazoria,3/20/2020,3
Brazos,3/20/2020,2
Cameron,3/20/2020,1
Collin,3/20/2020,12
Crane,3/20/2020,1
Dallas,3/20/2020,22


,county,num_cases
0,Bell,2
1,Bexar,12
2,Bowie,1
3,Brazoria,3
4,Brazos,2
5,Cameron,1
6,Collin,12
7,Crane,1
8,Dallas,22
9,Denton,6
